In [ ]:
# Запустите эту ячейку, чтобы загрузить графические пакеты

using Plots; gr()
using Interact

## Добавление параметра функции

В последнем блокноте мы увидели пример добавления **параметров** к функциям. Это значения, которые управляют поведением функции. Давайте посмотрим на это более подробно.

Давайте вернемся к нашей исходной версии функции σ:

In [ ]:
σ(x) = 1 / (1 + exp(-x))

Вместо того, чтобы работать с одной функцией, мы можем работать с целым классом (набором) функций, которые выглядят одинаково, но отличаются значением **параметра**. Давайте создадим новую функцию, которая использует предыдущую функцию $ \sigma $, но также имеет параметр $ w $. Математически мы могли бы написать

$$f_w(x) = f(x; w) = \sigma(w \, x).$$

(Здесь $ w $ и $ x $ умножаются на аргумент $ \sigma $; мы можем написать $ w \times x $ или $ w * x $, или даже $ w \cdot x $, но обычно это символы не пишутся.) 

Математически говоря, мы можем думать о $ f_w $ как о отдельной функции для каждого отдельного значения параметра $ w $.

В Julia мы пишем это следующим образом:

In [ ]:
f(x, w) = σ(w * x)

Обратите внимание, что Джулия просто рассматривает параметры как дополнительные *аргументы* для функции; функция `f` имеет два аргумента: значение` x` и значение `w`, которое мы хотим использовать.

Теперь мы можем исследовать эффект $ w $ в интерактивном режиме. Для этого нам нужен способ написать в Julia «функцию одной переменной $ x $, которую мы получаем, когда фиксируем значение $ w $». Мы пишем это как «анонимную функцию», как мы видели в блокноте о функциях:
```julia
    x -> f(x, w)
```
Мы можем прочитать это как «функцию, которая отображает $ x $ в значение $ f (x, w) $, для значения $ w $, которое было ранее задано».

Теперь мы готовы нарисовать функцию. Для каждого графика мы *фиксируем* значение параметра $ w $ и рисуем полученную функцию как функцию от $ x $. Однако `Interact.jl` позволяет нам интерактивно изменять значение $ w $ и выводить на экран новую функцию:

In [ ]:
@manipulate for w in -2:0.01:2
    
    plot(x->f(x, w), -5, 5, ylims=(0,1), label="sigmoid")
    plot!(x->(x>0), -5,5, label="Square Wave")
end

#### Упражнение 1

Попробуйте написать свою собственную функцию, которая принимает параметр. Начните с копирования и выполнения

```julia
square(x) = x^2
```

Затем используйте `square`, чтобы объявить новую функцию` square_and_scale`, которая принимает два входа: `a` и` x`, так что

$$\mathrm{square\_and\_scale}(x; a) := a \cdot x^2$$

#### Упражнение 2

После того как вы объявили `square_and_scale`, раскомментируйте приведенный ниже код и посмотрите, как параметр` a` масштабирует функцию `square`:

In [ ]:
# x = -10:10
# @manipulate for a in 0:0.01:10
#     plot(x, square.(x), label="x^2")
#     plot!(x, square_and_scale.(x, a), ls=:dash, label="ax^2")
# end

## Подгонка функции к данным

Как вы наверно помните, мы хотели бы использовать тот факт, что теперь у нас есть параметр в нашей функции, чтобы сделать что-то полезное! А именно, мы хотим смоделировать данные с его помощью.

Итак, предположим, что нам дана единственная точка данных $ (x_0, y_0) = (2, 0.8) $. Мы можем попытаться «подогнать» функцию $ f_w $, настраивая параметр $ w $, пока функция не пройдет через данные. 

**Игра**:  
перемещайте ползунок, пока график функции не достигнет точки данных. Какому значению $ w $ это соответствует?

In [ ]:
x0, y0 = 2, 0.8

@manipulate for w in -2:0.01:2
    plot(x->f(x, w), -5, 5, ylims=(0, 1), label="f")
    scatter!([x0], [y0], label="data")
end

## Количественная оценка того, как далеко мы от цели: *функция потерь*

Мы видим наглядно, когда график функции проходит через точку данных. Но цель состоит в том, чтобы иметь возможность автоматизировать этот процесс, чтобы компьютер мог делать это без посторонней помощи. 

Так что нам понадобится более точный способ определения и количественной оценки (то есть измерения с помощью числа) *как далеко мы от цели*; здесь цель означает попадание в точку данных с помощью функции.

#### Упражнение 3

Можете ли вы придумать способ измерения того, как далека наша функция от точки данных?

### Определение функции потерь

Нам нужно измерить, как далеко кривая находится от точки данных, когда мы выбираем конкретное значение $ w $. Один из способов сделать это - найти вертикальное расстояние $ d $ от кривой до точки данных. 

Вместо того, чтобы просто брать расстояние, обычно берется *квадрат* расстояния, $ d ^ 2 $. Поскольку мы берем вертикальное расстояние, нам нужно расстояние при заданном значении $ x_0 $, где лежит точка данных. Для данного значения параметра $ w $ высота точки на кривой с этим значением $ x_0 $ равна $ f (x_0, w) $.

Итак, мы берем
$$d := y_0 - f(x_0, w)$$

и
$$d^2 = [y_0 - f(x_0, w)]^2.$$

Это наша мера расстояния. Она изменяется, вместе с $ w $ - другими словами, она сама является *функцией $ w $*; мы будем обозначать эту функцию как $ L (w) $ и назовем ее **функцией потерь**:

$$L(w) := [y_0 - f(x_0, w)]^2.$$

Таким образом, цель состоит в том, чтобы найти значение $ w ^ * $ для $ w $, где функция потерь *наименьшая*; другими словами, нам нужно *минимизировать* функцию потерь! 

(Другое название функции потерь - *функция стоимости* или *оценочная функция*.)

#### Упражнение 4

(a) Определите функцию потерь `L (w)`.

(b) Нарисуйте точку данных и функцию `x -> f (x, w)`. Также нарисуйте вертикальную линию от точки данных к функции `x -> f(x, w)`.

(c) Сделайте сюжет интерактивным.

(d) Добавьте в качестве заголовка графика значение функции потерь для текущего значения $ w $.

(e) Используйте ползунок, чтобы найти значение $ w ^ * $ of $ w $, для которого функция потерь достигает своего минимального значения. Что такое $ w ^ * $? Каково значение функции потерь, $L(w^*)$?

## Как выглядит функция потерь?

Функция потерь $ L (w) $ сообщает нам, насколько далеко функция $ f_w $ от данных, когда значение параметра равно $ w $, визуально представленное в виде вертикальной линии на предыдущем графике. Когда данные фиксированы, это функция зависит только от параметра $ w $. Как эта функция выглядит как функция от $ w $? Давайте нарисуем это!

#### Упражнение 5

Нарисуйте функцию $ L (w) $ как функцию от $ w $.

### Особенности функции потерь

Этот график количественно показывает, как далеко мы находимся от точки данных для данного значения $ w $. Какие особенности мы видим на графике? 

Во-первых, мы видим, что $ L (w) $ всегда больше, чем $ 0 $, для любого значения $ w $. Это потому, что мы хотим, чтобы $ L $ была некоторой мерой *расстояния*, а расстояния не могут быть отрицательными. 

Во-вторых, мы видим, что существует специальное значение $ w ^ * $ для $ w $, где функция $ L $ достигает своего минимального значения. В данном конкретном случае он фактически достигает нуля! Это означает, что исходная функция $ f $ (та, которой мы манипулировали выше) проходит точно через точку данных $ (x_0, y_0) $.

#### Упражнение 6

Нарисуйте увеличенную версию графика, чтобы найти место $ w ^ * $, где функция точнее достигает $ 0 $.

### Другой способ определения функции потерь

**Почему мы использовали такую сложную функцию $ L $ с этими квадратами внутри?** Вместо этого мы могли бы просто использовать абсолютное расстояние вместо квадрата расстояния, используя функцию *absolute value*, записанную математически как $ | \cdot | $, а у Юлии как `abs`.

#### Упражнение 7 

Определите новую функцию потерь `L_abs`, используя абсолютное значение, и посмотрите, как она выглядит.